In [1]:
require 'hdf5'
require 'nn'

In [312]:
-- Loading data
myFile = hdf5.open('../data/MM_data.hdf5','r')
data = myFile:all()
input_matrix_train = data['input_matrix_train']
input_matrix_train_embed = data['input_matrix_train_embed']
input_matrix_dev = data['input_matrix_dev']
input_matrix_train_dev = data['input_matrix_dev_embed']
input_matrix_test = data['input_matrix_test']
myFile:close()

In [126]:
myFile = hdf5.open('../data/embeddings.hdf5','r')
data2 = myFile:all()
embeddings = data2['embeddings']
myFile:close()

In [313]:
train_output = input_matrix_train_embed:narrow(2,60,1)
nwords = train_output:size(1)
train_output = train_output:narrow(1,2,nwords-1)

In [318]:
train_input = torch.Tensor(nwords-1,10)
train_input:narrow(2,1,1):copy(input_matrix_train:narrow(2,1,1):narrow(1,2,nwords-1))
train_input:narrow(2,2,9):copy(input_matrix_train:narrow(2,2,9):narrow(1,1,nwords-1))

In [319]:
train_input:narrow(1,1,5)

  645     0     0     0     0     0     0     0     1     0
 7579     0     0     0     1     0     0     0     0     0
  515     1     0     0     0     0     0     0     0     0
  581     0     0     0     0     1     0     0     0     0
    5     1     0     0     0     0     0     0     0     0
[torch.DoubleTensor of size 5x10]



In [320]:
train_output:narrow(1,1,5)

 4
 1
 5
 1
 1
[torch.DoubleTensor of size 5x1]



### Linear 1:

In [156]:
linear = nn.Sequential()
linear:add(nn.Linear(59,9))
linear:add(nn.SoftMax())

In [157]:
criterion = nn.ClassNLLCriterion()

In [159]:
train_model(train_input, train_output, linear, criterion, 59, 9, 0.01, 20, 16);

Epoch 1: 0.60036301612854	
Average Loss: -0.66707073234158	


Epoch 2: 0.59119391441345	
Average Loss: -0.71237675728118	


Epoch 3: 0.59684014320374	
Average Loss: -0.72320236849443	


Epoch 4: 0.590656042099	


Average Loss: -0.73588772172317	


...

Epoch 16: 0.70232391357422	
Average Loss: -0.76065918617728	


Epoch 17: 0.64208316802979	
Average Loss: -0.76106153387546	


Epoch 18: 0.73686122894287	
Average Loss: -0.76142982197363	


Epoch 19: 0.56427597999573	
Average Loss: -0.7617710446001	


Epoch 20: 0.59289908409119	
Average Loss: -0.76209164453552	


### Linear 2:

In [325]:
function train_model(train_input, train_output, model, criterion, din, nclass, eta, nEpochs, batchSize)
    -- Train the model with a mini batch SGD
    -- standard parameters are
    -- nEpochs = 1
    -- batchSize = 32
    -- eta = 0.01

    -- To store the loss
    av_L = 0

    -- Memory allocation
    inputs_batch = torch.DoubleTensor(batchSize, din)
    targets_batch = torch.DoubleTensor(batchSize)
    outputs = torch.DoubleTensor(batchSize, nclass)
    df_do = torch.DoubleTensor(batchSize, nclass)

    for i = 1, nEpochs do
        -- timing the epoch
        timer = torch.Timer()
        av_L = 0
        
        -- mini batch loop
        for t = 1, train_input:size(1), batchSize do
            -- Mini batch data
            current_batch_size = math.min(batchSize,train_input:size(1)-t)
--             print('here1')
            
            inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
--             print('here2')
            
            targets_batch:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
--             print('here3')
            
            -- reset gradients
            model:zeroGradParameters()
            --gradParameters:zero()

            -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
            outputs:narrow(1,1,current_batch_size):copy(model:forward({inputs_batch:narrow(1,1,current_batch_size):narrow(2,1,1),
                                                                        inputs_batch:narrow(1,1,current_batch_size):narrow(2,2,9)}))
--             print('here4')
            -- Average loss computation
            f = criterion:forward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size))
            
--             print('here5')
            av_L = av_L +f

            -- Backward pass
            df_do:narrow(1,1,current_batch_size):copy(criterion:backward(outputs:narrow(1,1,current_batch_size), targets_batch:narrow(1,1,current_batch_size)))
--             print('here6')
            model:backward({inputs_batch:narrow(1,1,current_batch_size):narrow(2,1,1), inputs_batch:narrow(1,1,current_batch_size):narrow(2,2,9)}, 
                            df_do:narrow(1,1,current_batch_size))
--             print('here7')
            model:updateParameters(eta)
            
        end
            
        print('Epoch '..i..': '..timer:time().real)
        print('Average Loss: '..av_L/math.floor(train_input:size(1)/batchSize))
       
    end
    
end

In [326]:
LT = nn.LookupTable(400002,embed_dim)
LT.weight:narrow(1, 1, 400000):copy(embeddings)

In [327]:
ultimate_t = nn.Sequential()
t1 = nn.ParallelTable()

t1_1 = nn.Sequential()
t1_1:add(LT)
t1_1:add(nn.Squeeze())

t1_2 = nn.Identity()

t1:add(t1_1)
t1:add(t1_2)

ultimate_t:add(t1)
ultimate_t:add(nn.JoinTable(2))

ultimate_t:add(nn.Linear(59,9))
ultimate_t:add(nn.LogSoftMax())

In [328]:
-- Sanity check:
ultimate_t:forward{train_input:narrow(1,1,2):narrow(2,1,1),train_input:narrow(1,1,2):narrow(2,2,9)}

-2.3989 -2.1051 -1.9038 -2.0907 -1.9235 -2.1626 -3.1964 -1.9366 -2.6807
-1.9902 -1.7492 -2.8625 -2.1755 -1.7534 -2.2611 -3.0452 -2.1606 -2.5448
[torch.DoubleTensor of size 2x9]



In [329]:
train_model(train_input, train_output, ultimate_t, criterion, 10, 9, 0.01, 1, 32)

Epoch 1: 80.039309024811	
Average Loss: 0.53934032531741	



In [330]:
train_model(train_input, train_output, ultimate_t, criterion, 10, 9, 0.01, 10, 32)

Epoch 1: 82.327497959137	
Average Loss: 0.38857311932908	


Epoch 2: 82.844657897949	
Average Loss: 0.35317356242269	


Epoch 3: 80.959836006165	
Average Loss: 0.33439557057874	


Epoch 4: 81.129220962524	
Average Loss: 0.32182561834837	


Epoch 5: 77.538828849792	
Average Loss: 0.31238966997968	


Epoch 6: 72.771478891373	
Average Loss: 0.30483280008797	


Epoch 7: 73.385397911072	
Average Loss: 0.29852969310718	


Epoch 8: 72.443921804428	
Average Loss: 0.29312352643758	


Epoch 9: 72.708081007004	
Average Loss: 0.2883902760815	


Epoch 10: 73.55508184433	
Average Loss: 0.28417927370669	



In [331]:
train_model(train_input, train_output, ultimate_t, criterion, 10, 9, 0.01, 10, 32)

Epoch 1: 72.742295980453	
Average Loss: 0.28038393612744	


Epoch 2: 72.818717956543	
Average Loss: 0.27692597167988	


Epoch 3: 73.505111932755	
Average Loss: 0.27374618046984	


Epoch 4: 72.681257963181	
Average Loss: 0.27079876665041	


Epoch 5: 72.52105307579	
Average Loss: 0.26804764026582	


Epoch 6: 77.478487014771	
Average Loss: 0.26546391665212	


Epoch 7: 72.745879888535	
Average Loss: 0.26302417481225	


Epoch 8: 72.614097833633	
Average Loss: 0.26070920098141	


Epoch 9: 73.487210035324	
Average Loss: 0.25850307927189	


Epoch 10: 72.5062520504	
Average Loss: 0.25639248668346	

